<h1>Libraries</h1>

In [381]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

%matplotlib inline

<h1>Mounting the Drive</h1>

In [382]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1>Dataset Operations</h1>

In [383]:
def build_dataset():
    training_set=h5py.File('/content/drive/My Drive/training_set.h5','r')
    X=training_set['inputs'][:]
    Y=training_set['labels'][:]
    training_set.close()
    return X,Y

def build_mini_batches(X,Y,size):
    X=X.reshape(X.shape[0],X.shape[1],X.shape[2])
    X=X.reshape(X.shape[0],X.shape[1]*X.shape[2])
    X=X.T
    m=X.shape[1]
    assert size<=m
    permutation=np.random.permutation(m)
    X=X[:,permutation]
    Y=Y[:,permutation]
    mini_batches=[]
    whole_batches=m//size
    for t in range(whole_batches):
        X_mini_batch=X[:,t*size:(t+1)*size]
        Y_mini_batch=Y[:,t*size:(t+1)*size]
        mini_batches.append((X_mini_batch,Y_mini_batch))
    if m%size!=0:
        X_mini_batch=X[:,whole_batches*size:m]
        Y_mini_batch=Y[:,whole_batches*size:m]
        mini_batches.append((X_mini_batch,Y_mini_batch))
    return mini_batches

def reshuffle_split_mini_batches(X,Y,hp):
    mini_batches=build_mini_batches(X,Y,hp['mini_batch_size'])
    split_mini_batches=[]
    for (Xt,Yt) in mini_batches:
        split_mini_batches.append((split_into_strips(Xt,hp['strips']),Yt))
    return split_mini_batches

def split_into_strips(Xt,strips):
    if int(np.sqrt(Xt.shape[0]))%strips==0:
        width=Xt.shape[0]//strips
        split_mini_batch=[]
        for i in range(strips):
            split_mini_batch.append(Xt[i*width:(i+1)*width,:])
        return tuple(split_mini_batch)
    else:
        raise ValueError('you have elected a number of strips that results in strips of unequal width. please re-enter the number of strips you\'d like, and try again.')

<h1>Initialization Operations</h1>

In [384]:
def initialize_parameters(layer_dims,initialization):
    L=len(layer_dims)
    parameters={}
    if initialization=='None':
        for l in range(1,L):
            parameters['W'+str(l)]=0.01*np.random.rand(layer_dims[l],layer_dims[l-1])
            parameters['gamma'+str(l)]=np.ones((layer_dims[l],1))
            parameters['beta'+str(l)]=np.zeros((layer_dims[l],1))
    elif initialization=='He':
        for l in range(1,L):
            const=np.sqrt(2/layer_dims[l-1])
            parameters['W'+str(l)]=const*np.random.rand(layer_dims[l],layer_dims[l-1])
            parameters['gamma'+str(l)]=np.ones((layer_dims[l],1))
            parameters['beta'+str(l)]=np.zeros((layer_dims[l],1))
    elif initialization=='Xavier':
        for l in range(1,L):
            const=np.sqrt(1/layer_dims[l-1])
            parameters['W'+str(l)]=const*np.random.rand(layer_dims[l],layer_dims[l-1])
            parameters['gamma'+str(l)]=np.ones((layer_dims[l],1))
            parameters['beta'+str(l)]=np.zeros((layer_dims[l],1))
    elif initialization=='Other':
        for l in range(1,L):
            const=np.sqrt(2/(layer_dims[l]+layer_dims[l-1]))
            parameters['W'+str(l)]=const*np.random.rand(layer_dims[l],layer_dims[l-1])
            parameters['gamma'+str(l)]=np.ones((layer_dims[l],1))
            parameters['beta'+str(l)]=np.zeros((layer_dims[l],1))
    return parameters

def build_nets(split_mini_batch,hp):
    X=split_mini_batch[0][0]
    Y=split_mini_batch[1]
    subnet_parameters={}
    if hp['subnet_hidden']==[]:
        layer_dims=[X.shape[0],Y.shape[0]]
    else:
        layer_dims=[X.shape[0]]+hp['subnet_hidden']+[Y.shape[0]]
    for i in range(hp['strips']):
        subnet_parameters['subnet'+str(i+1)]=initialize_parameters(layer_dims,hp['initialization'])
    if hp['supnet_hidden']==[]:
        layer_dims=[hp['strips']*Y.shape[0],Y.shape[0]]
    else:
        layer_dims=[hp['strips']*Y.shape[0]]+hp['supnet_hidden']+[Y.shape[0]]
    supnet_parameters=initialize_parameters(layer_dims,hp['initialization'])
    return subnet_parameters,supnet_parameters

def normalize_inputs(X,epsilon=1e-8):
    return X/255


<h1>Network Operations</h1>

In [385]:
def sigmoid(z,backward=False):
    if backward:
        return (1/(1+np.exp(-z)))*(1-(1/(1+np.exp(-z))))
    else:
        return 1/(1+np.exp(-z))

def relu(z,backward=False):
    if backward:
        return np.where(z<0,0,1)
    else:
        return np.where(z<0,0,z)

def softmax(z):
    magnitude=np.sum(np.exp(z),axis=0,keepdims=True)
    return np.exp(z)/magnitude

def concat(z,classes,m=None):
    if m==None:
        subnet_dA=[]
        for i in range(z.shape[0]//classes):
            subnet_dA.append(z[i*classes:(i+1)*classes,:])
        return subnet_dA
    else:
        new_z=np.zeros((len(z)*classes,m))
        for i in range(len(z)):
            new_z[i*classes:(i+1)*classes,:]=z[i]
        return new_z

def propagate_forward(X,parameters,activations,epsilon=1e-8):
    A_prev=X
    m=X.shape[1]
    cache_list=[]
    L=len(parameters)//3
    for l in range(L):
        Z=np.dot(parameters['W'+str(l+1)],A_prev)
        mu=np.mean(Z,axis=1,keepdims=True)
        std=np.var(Z,axis=1,keepdims=True)
        Zhat=(Z-mu)/np.sqrt(std+epsilon)
        Ztilde=parameters['gamma'+str(l+1)]*Zhat+parameters['beta'+str(l+1)]
        cache_list.append((Ztilde,Zhat,mu,Z,std,A_prev))
        A_prev=activations[l](Ztilde)
    return cache_list,A_prev

def compute_cost(AL,Y):
    m=Y.shape[1]
    loss=-np.sum(Y*np.log(AL),axis=0,keepdims=True)
    return np.squeeze(np.sum(loss,axis=1,keepdims=True))/m

def propagate_backward(Y,AL,cache_list,parameters,activations,epsilon=1e-8):
    grads={}
    m=Y.shape[1]
    L=len(parameters)//3
    for l in reversed(range(L)):
        (Ztilde,Zhat,mu,Z,std,A_prev)=cache_list[l]
        if l==L-1:
            dZtilde_loss=AL-Y
            dZtilde=dZtilde_loss/m
        else:
            dZtilde=dA*activations[l](Ztilde,backward=True)
        dZhat=dZtilde*parameters['gamma'+str(l+1)]
        dgamma=np.sum(dZtilde*Zhat,axis=1,keepdims=True)
        dbeta=np.sum(dZtilde,axis=1,keepdims=True)
        dstd=np.sum(dZhat*((mu-Z)/(2*(std+epsilon)**(3/2))),axis=1,keepdims=True)
        dmu=-np.sum(dZhat*(1/np.sqrt(std+epsilon)),axis=1,keepdims=True)
        dZ=dZhat*(1/np.sqrt(std+epsilon))+(2*dstd*(Z-mu))/m+dmu/m
        dW=np.dot(m*dZ,A_prev.T)
        dA=np.dot(parameters['W'+str(l+1)].T,dZ)
        grads['dgamma'+str(l+1)]=dgamma
        grads['dbeta'+str(l+1)]=dbeta
        grads['dW'+str(l+1)]=dW
    return grads

def update_parameters(parameters,grads,learning_rate,epoch_num):
    lrate=learning_rate/(1+(1e-1)*epoch_num)
    L=len(parameters)//3
    for l in range(L):
        parameters['W'+str(l+1)]=parameters['W'+str(l+1)]-lrate*grads['dW'+str(l+1)]
        parameters['gamma'+str(l+1)]=parameters['gamma'+str(l+1)]-lrate*grads['dgamma'+str(l+1)]
        parameters['beta'+str(l+1)]=parameters['beta'+str(l+1)]-lrate*grads['dbeta'+str(l+1)]
    return parameters

<h1>Compilation Operation(s)</h1>

In [386]:
def train(X,Y,hp):
    normalized_X=normalize_inputs(X)
    split_mini_batches=reshuffle_split_mini_batches(normalized_X,Y,hp)
    subnet_parameters,supnet_parameters=build_nets(split_mini_batches[0],hp)
    for e in range(hp['epochs']):
        for (Xt,Yt) in split_mini_batches:
            all_subnet_AL=[]
            all_caches={}
            for i in range(len(Xt)):
                subnet_cache,subnet_AL=propagate_forward(Xt[i],subnet_parameters['subnet'+str(i+1)],hp['subnet_activations'])
                all_subnet_AL.append(subnet_AL)
                all_caches['subnet'+str(i+1)]=subnet_cache
                subnet_grads=propagate_backward(Yt,subnet_AL,subnet_cache,subnet_parameters['subnet'+str(i+1)],hp['subnet_activations'])
                subnet_parameters['subnet'+str(i+1)]=update_parameters(subnet_parameters['subnet'+str(i+1)],subnet_grads,hp['learning_rate'],e)
            Xt_concat=concat(all_subnet_AL,Yt.shape[0],m=Yt.shape[1])
            supnet_cache,supnet_AL=propagate_forward(Xt_concat,supnet_parameters,hp['supnet_activations'])
            all_caches['supnet']=supnet_cache
            supnet_grads=propagate_backward(Yt,supnet_AL,supnet_cache,supnet_parameters,hp['supnet_activations'])
            supnet_parameters=update_parameters(supnet_parameters,supnet_grads,hp['learning_rate'],e)
        Y_hat=np.where(supnet_AL==np.amax(supnet_AL,axis=0,keepdims=True),1,0)
        print('--------------------------------------------------')
        print('Cost after epoch '+str(e+1)+': '+str(compute_cost(supnet_AL,Yt)))
        print('Error on final mini batch in this epoch: '+str(100*(np.linalg.norm(Yt-Y_hat)/np.sqrt(2*Yt.shape[1]))))
        print('--------------------------------------------------')
        split_mini_batches=reshuffle_split_mini_batches(normalized_X,Y,hp)
    return subnet_parameters,supnet_parameters,all_caches

<h1>Inference Operations</h1>

In [387]:
def to_inf_params(parameters,cache_list,epsilon=1e-8):
    inf_params={}
    L=len(parameters)//3
    for l in range(L):
        (Ztilde,Zhat,mu,Z,std,A_prev)=cache_list[l]
        inf_params['W'+str(l+1)]=parameters['W'+str(l+1)]
        inf_params['gamma'+str(l+1)]=parameters['gamma'+str(l+1)]/np.sqrt(std+epsilon)
        inf_params['beta'+str(l+1)]=parameters['beta'+str(l+1)]-mu*(parameters['gamma'+str(l+1)]/np.sqrt(std+epsilon))
    return inf_params

def inference_forward(X,parameters,activations,epsilon=1e-8):
    A_prev=X
    L=len(parameters)//3
    for l in range(L):
        Z=np.dot(parameters['W'+str(l+1)],A_prev)
        Ztilde=parameters['gamma'+str(l+1)]*Z+parameters['beta'+str(l+1)]
        A_prev=activations[l](Ztilde)
    return A_prev

def perc_error(X,Y,subnet_parameters,supnet_parameters,hp):
    normalized_X=normalize_inputs(X)
    [(Xt,Yt)]=reshuffle_split_mini_batches(normalized_X,Y,{'mini_batch_size':Y.shape[1],'strips':hp['strips']})
    all_subnet_AL=[]
    for i in range(len(Xt)):
        subnet_AL=inference_forward(Xt[i],subnet_parameters['subnet'+str(i+1)],hp['subnet_activations'])
        all_subnet_AL.append(subnet_AL)
    Xt_concat=concat(all_subnet_AL,Yt.shape[0],m=Yt.shape[1])
    supnet_AL=inference_forward(Xt_concat,supnet_parameters,hp['supnet_activations'])
    Y_hat=np.where(supnet_AL==np.amax(supnet_AL,axis=0,keepdims=True),1,0)
    assert Y_hat.shape==Y.shape
    return 100*(np.linalg.norm(Yt-Y_hat)/np.sqrt(2*Yt.shape[1]))

<h1>FruitNet API</h1>

In [388]:
hyperparameters={'learning_rate':0.05,
                 'epochs':256,
                 'mini_batch_size':128,
                 'initialization':'Other',
                 'strips':int(4),
                 'subnet_hidden':[128],
                 'subnet_activations':[relu,softmax],
                 'supnet_hidden':[],
                 'supnet_activations':[softmax]}

class FruitNet():

    def __init__(self):

        #instantiate hyperparameters
        self.__hp={'learning_rate':0,
                   'epochs':0,
                   'mini_batch_size':0,
                   'initialization':None,
                   'strips':0,
                   'subnet_names':[],
                   'subnet_hidden':[],
                   'subnet_activations':[],
                   'supnet_names':[],
                   'supnet_hidden':[],
                   'supnet_activations':[]}
    
    def __raw_shuffle__(self,X,Y):

        #permute [1,2,...,m]
        permutation=np.random.permutation(X.shape[0])

        #shuffle X and Y in accordance with the above permutation
        shuffled_X=X[permutation,:,:,:]
        shuffled_Y=Y[:,permutation]

        return shuffled_X,shuffled_Y

    def loadData(self,filename,features,labels,split=0.05):

        #retrieve data
        try:
            training_set=h5py.File(filename,'r')
            X=training_set[features][:]
            Y=training_set[labels][:]
            training_set.close()
        except:
            raise NameError('you\'ve entered an incorrect h5py filename, feature dataset name, or label dataset name, when loading in your data.')

        #confirm validity of X dataset shape
        assert len(X.shape)==4
        assert X.shape[3]==1

        #confirm validity of Y dataset shape
        assert len(Y.shape)==2
        assert X.shape[0]==Y.shape[1]

        #shuffle datasets (maintaining correspondence)
        shuffled_X,shuffled_Y=self.__raw_shuffle__(X,Y)

        #establish cross-validation test set (and, therefore, training set) sizes
        m=X.shape[0]
        self.__test_quant=int(split*m)
        self.__train_quant=m-self.__test_quant

        #create cross-validation test set and training set using the above sizes
        self.__X_train=shuffled_X[self.__test_quant:self.__test_quant+self.__train_quant,:,:,:]
        self.__Y_train=shuffled_Y[:,self.__test_quant:self.__test_quant+self.__train_quant]
        self.__X_test=shuffled_X[:self.__test_quant,:,:,:]
        self.__Y_test=shuffled_Y[:,:self.__test_quant]

    def __retrieve_single_split_example__(self,strips,view_data=True):

        #select a random sample number
        try:
            sample=np.random.randint(0,self.__test_quant)
        except:
            raise UnboundLocalError('self.__test_quant has referenced before assignment, meaning you have not yet loaded in any data. please load in data and try again.')

        #select the sample from the test set and split into regions
        X=np.zeros((1,self.__X_test.shape[1],self.__X_test.shape[2],1))
        Y=np.zeros((self.__Y_test.shape[0],1))
        X[0,:,:,0]=self.__X_test[sample,:,:,0]
        Y[:,0]=self.__Y_test[:,sample]
        [(Xt,Yt)]=reshuffle_split_mini_batches(X,Y,{'mini_batch_size':1,'strips':strips})

        #check why the function has been called and return accordingly
        if view_data:
            return X,Xt,Y
        else:
            return (Xt,Yt)


    def viewData(self,strips):

        #confirm the validity of the number of strips
        assert type(strips)==int
        assert strips>1

        #retrieve (split) item of data (from test set) corresponding to random_sample
        X,Xt,Y=self.__retrieve_single_split_example__(strips)

        #plot the sample, as well as the strips into which we've split it, and the corresponding label
        fig=plt.figure()
        axes=fig.subplots(strips+1,1)
        axes[0].imshow(X[0,:,:,0])
        for i in range(strips):
            axes[i+1].imshow(Xt[i].reshape(X.shape[1]//strips,X.shape[2]))
        print('The label corresponding to the above item of data (read from left to right) is given by:\n'+str(Y.reshape(Y.shape[0],)))
    
    def addLayer(self,name,n_H,activation,net='sub'):
        
        #confirm the validity of name, n_H, activation, and supnet
        assert type(name)==str
        assert type(n_H)==int
        assert n_H>0
        assert activation in [relu,sigmoid]
        assert type(net)==str

        #append the hyperparameters in accordance with the function parameters
        if net=='sup':
            assert name not in self.__hp['supnet_names']
            self.__hp['supnet_names'].append(name)
            self.__hp['supnet_hidden'].append(n_H)
            self.__hp['supnet_activations'].append(activation)
        elif net=='sub':
            assert name not in self.__hp['subnet_names']
            self.__hp['subnet_names'].append(name)
            self.__hp['subnet_hidden'].append(n_H)
            self.__hp['subnet_activations'].append(activation)
        else:
            raise ValueError('you have not selected a valid network to add a layer to. please re-enter the net string and try again.')
    
    def __decoy_parameters__(self):

        #extract an example from the test set to preserve computational efficiency
        single_example_mini_batch=self.__retrieve_single_split_example__(self.__hp['strips'],view_data=False)

        #generate and return relevant parameters (NOT as class attributes, though)
        return build_nets(single_example_mini_batch,self.__hp)

    def modelSummary(self):

        #confirm that enough hyperparameters have been specified in order for a summary to be generated
        assert self.__hp['strips']>0

        #retrieve the parameters whose summary we wish to produce
        try:
            subnet=self.__subnet_parameters['subnet1']
            supnet=self.__supnet_parameters
        except:
            subnets,supnet=self.__decoy_parameters__()
            subnet=subnets['subnet1']
        
        #produce model summary header
        print('==================================================')
        print('NETWORK ARCHITECTURE SUMMARY')
        print('==================================================\n\n')

        #produce subnet summary
        print('SUBNET SUMMARY')
        print('--------------------------------------------------')
        for i in range(len(subnet)//3+1):
            if i==0:
                print('Input Layer:')
                print('- Name: N/A')
                print('- No. of Input Nodes: '+str(subnet['W1'].shape[1]))
                print('- Activation Function: N/A')
                print('--------------------------------------------------')
            else:
                if i==len(subnet)//3:
                    print('Output Layer:')
                    print('- Name: N/A')
                    print('- No. of Output Nodes: '+str(subnet['W'+str(i)].shape[0]))
                    print('Activation Function: softmax')
                    print('--------------------------------------------------')
                else:
                    print('Hidden Layer '+str(i)+':')
                    print('- Name: '+str(self.__hp['subnet_names'][i-1]))
                    print('- No. of Hidden Nodes: '+str(subnet['W'+str(i)].shape[0]))
                    if self.__hp['subnet_activations'][i-1]==relu:
                        print('- Activation Function: relu')
                    else:
                        print('- Activation Function: sigmoid')
                    print('--------------------------------------------------')
        print('\n')

        #produce supnet summary
        print('SUPNET SUMMARY')
        print('--------------------------------------------------')
        for i in range(len(supnet)//3+1):
            if i==0:
                print('Input Layer:')
                print('- Name: N/A')
                print('- No. of Input Nodes: '+str(supnet['W1'].shape[1]))
                print('- Activation Function: N/A')
                print('--------------------------------------------------')
            else:
                if i==len(supnet)//3:
                    print('Output Layer:')
                    print('- Name: N/A')
                    print('- No. of Output Nodes: '+str(supnet['W'+str(i)].shape[0]))
                    print('Activation Function: softmax')
                    print('--------------------------------------------------')
                else:
                    print('Hidden Layer '+str(i)+':')
                    print('- Name: '+str(self.__hp['supnet_names'][i-1]))
                    print('- No. of Hidden Nodes: '+str(supnet['W'+str(i)].shape[0]))
                    if self.__hp['supnet_activations'][i-1]==relu:
                        print('- Activation Function: relu')
                    else:
                        print('- Activation Function: sigmoid')
                    print('--------------------------------------------------')
        print('\n')

    def adjustLearningRate(self,learning_rate):

        #confirm the validity of the proposed learning_rate
        assert type(learning_rate)==float
        assert learning_rate>0

        #inform the user of the current learning rate, as well as the updated learning rate
        print('The current learning rate is '+str(self.__hp['learning_rate'])+'.')
        self.__hp['learning_rate']=learning_rate
        print('And the new learning rate is '+str(self.__hp['learning_rate'])+'.')

    def adjustEpochs(self,epochs):

        #confirm the validity of the proposed number of epochs
        assert type(epochs)==int
        assert epochs>0

        #inform the user of the current number of epochs, as well as the updated number of epochs
        print('The current number of epochs is '+str(self.__hp['epochs'])+'.')
        self.__hp['epochs']=epochs
        print('And the new number of epochs is '+str(self.__hp['epochs'])+'.')

    def adjustBatchSize(self,size):

        #confirm the validity of the proposed number of epochs
        assert type(size)==int
        assert size>0

        #inform the user of the current mini batch size, as well as the updated mini batch size
        print('The current mini batch size is '+str(self.__hp['mini_batch_size'])+'.')
        self.__hp['mini_batch_size']=size
        print('And the new mini batch size is '+str(self.__hp['mini_batch_size'])+'.')
    
    def adjustInitialization(self,init):

        #confirm the validity of the proposed intialization technique
        assert init in ['None','He','Xavier','Other']

        #inform the user of the current intialization technique, as well as the updated initialization technique
        print('The current intialization tehcnique is '+str(self.__hp['initialization'])+'.')
        self.__hp['initialization']=init
        print('And the new initialization technique is '+str(self.__hp['initialization'])+'.')

    def adjustStrips(self,strips):

        #confirm the validity of the proposed numbwe of strips
        assert type(strips)==int
        assert strips>1

        #inform the user of the current number of strips, as well as the updated number of strips
        print('The current number of strips into which training examples are split is '+str(self.__hp['strips'])+'.')
        self.__hp['strips']=strips
        print('And the new number of strips into which training examples are split is '+str(self.__hp['strips'])+'.')
    
    def compile(self):

        #check the validity of the relevant hyperparameters before initiating training
        assert self.__hp['learning_rate']>0
        assert self.__hp['epochs']>0
        assert self.__hp['mini_batch_size']>0
        assert self.__hp['strips']>1

        #expand the activation function hyperparameters for both the sub- and the supnet
        self.__hp['subnet_activations'].append(softmax)
        self.__hp['supnet_activations'].append(softmax)

        #exectute the relevant training
        try:
            self.__subnet_parameters,self.__supnet_parameters,inf_caches=train(self.__X_train,self.__Y_train,self.__hp)
        except:
            raise UnboundLocalError('self.__X_train and self.__Y_train referenced before assignment, meaning you have not loaded in any data. please load in a dataset and try again.')

        #instantiate the networks inference parameters
        self.__subnet_inf_parameters={}
        for i in range(len(self.__subnet_parameters)):
            self.__subnet_inf_parameters['subnet'+str(i+1)]=to_inf_params(self.__subnet_parameters['subnet'+str(i+1)],inf_caches['subnet'+str(i+1)])
        self.__supnet_inf_parameters=to_inf_params(self.__supnet_parameters,inf_caches['supnet'])

        #check the network's performance on the test set
        print('The network\'s performance on the test set yields an error of roughly '+str(perc_error(self.__X_test,self.__Y_test,self.__subnet_inf_parameters,self.__supnet_inf_parameters,self.__hp)))

        #rectify the activation function hyperparameters for both the sub- and the supnet once training is complete
        self.__hp['subnet_activations'].pop()
        self.__hp['supnet_activations'].pop()

    def saveModel(self):
        pass

<h1>FruitNet_v1.0</h1>

In [390]:
#instantiate the network
fruitnet_v1=FruitNet()

#load in data from (my own) Google Drive
fruitnet_v1.loadData('/content/drive/My Drive/training_set.h5','inputs','labels')

#add a hidden layer of size 128 with ReLU activation to the subnet
fruitnet_v1.addLayer('sub_hidden_1',128,relu)

#adjust each of the relevant hyperparameters
fruitnet_v1.adjustLearningRate(0.05)
fruitnet_v1.adjustEpochs(256)
fruitnet_v1.adjustBatchSize(128)
fruitnet_v1.adjustInitialization('Other')
fruitnet_v1.adjustStrips(4)

#produce and view a summary of the model
fruitnet_v1.modelSummary()

#train the model
fruitnet_v1.compile()

The current learning rate is 0.
And the new learning rate is 0.05.
The current number of epochs is 0.
And the new number of epochs is 256.
The current mini batch size is 0.
And the new mini batch size is 128.
The current intialization tehcnique is None.
And the new initialization technique is Other.
The current number of strips into which training examples are split is 0.
And the new number of strips into which training examples are split is 4.
NETWORK ARCHITECTURE SUMMARY


SUBNET SUMMARY
--------------------------------------------------
Input Layer:
- Name: N/A
- No. of Input Nodes: 196
- Activation Function: N/A
--------------------------------------------------
Hidden Layer 1:
- Name: sub_hidden_1
- No. of Hidden Nodes: 128
- Activation Function: relu
--------------------------------------------------
Output Layer:
- Name: N/A
- No. of Output Nodes: 10
Activation Function: softmax
--------------------------------------------------


SUPNET SUMMARY
---------------------------------